In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai import *
import torch
from fastai.vision import *
%matplotlib inline

In [ ]:
train_df = pd.read_csv("../input/train.csv")
train_df.head()


In [ ]:
test_df = pd.read_csv("../input/sample_submission.csv")
test_img = ImageList.from_df(test_df, path="../input", folder='test/test')
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)

In [ ]:
data = (ImageList.from_df(train_df,path="../input/train",folder="train")
        .split_by_rand_pct()
#         .split_none()
        .label_from_df()
        .add_test(test_img)
        .transform(tfms,size = 128)
        .databunch(path='.',bs=64)    
        .normalize(imagenet_stats)
       )

In [ ]:
data.show_batch(rows=3,figsize = (5,5))

In [ ]:
data.valid_ds.classes

In [ ]:
model = cnn_learner(data,models.resnet152, metrics = [accuracy])

In [ ]:
model.summary()

In [ ]:
lr = 1e-5
model.fit_one_cycle(5,lr)

In [ ]:
# model.unfreeze()
# model.fit_one_cycle(10)

In [ ]:
model.lr_find()
model.recorder.plot(suggestion = True)

In [ ]:
# lr = 1e-5
model.fit_one_cycle(10,max_lr=slice(1e-2,1e-4))

In [ ]:
model.fit_one_cycle(10,max_lr=slice(1e-2,1e-4))

In [ ]:
model.fit_one_cycle(10,max_lr=slice(1e-2,1e-4))

In [ ]:
model.save('stage-1-resnet152')

In [ ]:
model.recorder.plot_losses()

In [ ]:
interpreter = ClassificationInterpretation.from_learner(model)


In [ ]:
interpreter.plot_confusion_matrix()

In [ ]:
img = model.data.test_ds
img

In [ ]:
preds, _ = model.get_preds(ds_type=DatasetType.Test)
test_df.has_cactus = preds.numpy()[:,0]
test_df.to_csv('submission.csv', index=False)